<h3>Twitter APIs</h3>
<li>Sign in to the developer site using your twitter account
<li>Go to https://apps.twitter.com/app/new 
<li>Give your app a name and a description
<li>Enter anything for website (e.g., http://www.columbia.edu)
<li>Leave callback url blank
<li>Accept the terms and conditions and create an account
<li>Then, click on the "Keys and Access Tokens" tab
<li>Copy the two consumer keys to a text file
<li>Scroll down and click "Generate Access key and Secret"
<li>Copy the two access keys to the text file


In [1]:
with open("/Users/hardeepjohar/Documents/Courses/Fall2018/API_KEYS/tokens.txt",'r') as token_file:
    contents = token_file.read().split('\n')
    consumer_key = contents[0]
    consumer_secret = contents[1]
    access_token = contents[2]
    access_token_secret = contents[3]
    

<h2>tweepy</h2>
<li>A python library that interfaces with the twitter API
<li>Returns a lot of useful stuff but we'll only look at the tweet text

<h4>Set up an authentication object using OAuth and send a search request</h4>
<li>Tweepy constructs a "list like" SearchResults object
<li>Each item in SearchResults is a Status object
<li>The status object has a _json attribute that contains a json tweet
<li>http://docs.tweepy.org/en/v3.5.0/getting_started.html#

In [2]:
import tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
search_term = 'AAPL'
## fill in your search query and store your results in a variable
results = api.search(q = search_term, lang = "en", result_type = "recent", count = 1000)



In [3]:
print(type(results))
print(len(results))
print(type(results[0]))

<class 'tweepy.models.SearchResults'>
54
<class 'tweepy.models.Status'>


<li>We can save this to files that can then be read by a plaintextcorpus reader
<li>The order is important
<li>We could use  datetime for ordering
<li>But we'll just number them for now
<li>Saving to a file also helps us build a tweet corpus


In [4]:
search_term = 'AAPL'
for i in range(len(results)):
    fname = search_term+'.'+str(len(results)-i)
    with open('/Users/hardeepjohar/Documents/Courses/Fall2018/data/tweets/'+fname,'w') as f:
        f.write(results[i]._json['text']+'\n')

<h4>Now we can do sentiment analysis on these tweets</h4>

In [6]:
from nltk.corpus import PlaintextCorpusReader
apple_data = PlaintextCorpusReader('/Users/hardeepjohar/Documents/Courses/Fall2018/data/tweets/','AAPL.*')

In [7]:
apple_data.raw()

'AAPL: 220.7468 at Oct 24, 2018 4:23 PM\nToday\'s Information Technology market mover is CYBEROPTICS CORP\n$CYBE is up 6.48%!\nKey words: BUY, REPORT, POSITIVE… https://t.co/SzffblWZY7\nRT @FXpresso: $AAPL #apple "Jump" Edition\nLots going on here - lets break it down:\n- Double gaps to fill (pink)\n- Complex HS pattern comple…\nRT @myQuikfo: Today\'s Information Technology market mover is CYBEROPTICS CORP\n$CYBE is up 6.48%!\nKey words: BUY, REPORT, POSITIVE, RAISES,…\nToday\'s Consumer Staples market mover is REVLON INC\n$REV is up 4.53%!\nKey words: BUY, GOOD, BUYING, BOUGHT, DOWN, B… https://t.co/Tl6KCWiG9q\nToday\'s Financials market mover is MARKETAXESS HOLDINGS IN\n$MKTX is up 6.33%!\nKey words: CONFIDENT, UPGRADED, PROBL… https://t.co/OEKglO0mw7\nToday\'s Consumer Discretionary market mover is K12 INC\n$LRN is up 21.00%!\nKey words: EXPECTING, GROWTH, DOWN, BANKR… https://t.co/SyE4BNkB4v\nRT @myQuikfo: Today\'s Financials market mover is MARKETAXESS HOLDINGS IN\n$MKTX is up 6.3

In [10]:
def get_nrc_data():
    nrc = "/Users/hardeepjohar/Documents/Courses/Fall2018/data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count=0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        all_lines = list()
        for line in f:
            if count < 46:
                count+=1
                continue
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict


def emotion_analyzer(text,emotion_dict=get_nrc_data()):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y}
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0

    #Analyze the text and normalize by total number of words
    total_words = len(text.split())
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/len(text.split())
    return emotion_count

def comparative_emotion_analyzer(text_tuples,object_name="Restaurant",print_output=False):
    if print_output:
        print("%-20s %1s\t%1s %1s %1s %1s   %1s %1s %1s %1s"%(object_name,
                                                              "fear","trust","negative","positive",
                                                              "joy","disgust","anticip", "sadness",
                                                              "surprise"))
    import pandas as pd
    df = pd.DataFrame(columns=[object_name,'Fear','Trust','Negative',
                           'Positive','Joy','Disgust','Anticipation',
                           'Sadness','Surprise'],)
    df.set_index(object_name,inplace=True)
    
    output = df    
    for text_tuple in text_tuples:
        text = text_tuple[1] 
        result = emotion_analyzer(text)
        if print_output:
            print("%-20s %1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f"%(
                text_tuple[1][0:20],result['fear'],result['trust'],
                  result['negative'],result['positive'],result['joy'],result['disgust'],
                  result['anticipation'],result['sadness'],result['surprise']))
        df.loc[text_tuple[0]] = [result['fear'],result['trust'],
                  result['negative'],result['positive'],result['joy'],result['disgust'],
                  result['anticipation'],result['sadness'],result['surprise']]
    return output
#And test it        

In [11]:
comparative_emotion_analyzer([['apple',apple_data.raw()]],object_name='Apple Tweets',print_output=False)

,Fear,Trust,Negative,Positive,Joy,Disgust,Anticipation,Sadness,Surprise
Apple Tweets,,,,,,,,,
apple,0.006571,0.010155,0.011947,0.020311,0.004779,0.002389,0.008961,0.002987,0.003584


In [12]:
def get_tweets(search_term,consumer_key=consumer_key,consumer_secret=consumer_secret,
               access_token=access_token,access_token_secret=access_token_secret):
    import tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
#    search_term = 'AAPL'
    ## fill in your search query and store your results in a variable
    results = api.search(q = search_term, lang = "en", result_type = "recent", count = 1000)
#    search_term = 'AAPL'
    for i in range(len(results)):
        fname = search_term+'.'+str(len(results)-i)
        with open('/Users/hardeepjohar/Documents/Courses/Fall2018/data/tweets/'+fname,'w') as f:
            f.write(results[i]._json['text']+'\n')

get_tweets(search_term="GOOG")
            

In [14]:
import nltk
from nltk.corpus import PlaintextCorpusReader
tweets_root = "/Users/hardeepjohar/Documents/Courses/Fall2018/data/tweets"
apple_files = "AAPL.*"
google_files = "GOOG.*"
apple_data = PlaintextCorpusReader(tweets_root,apple_files)
google_data = PlaintextCorpusReader(tweets_root,google_files)
comparative_emotion_analyzer([
    ['apple',apple_data.raw()],
    ['google',google_data.raw()]],
    object_name='Equity',print_output=False)

,Fear,Trust,Negative,Positive,Joy,Disgust,Anticipation,Sadness,Surprise
Equity,,,,,,,,,
apple,0.006571,0.010155,0.011947,0.020311,0.004779,0.002389,0.008961,0.002987,0.003584
google,0.007864,0.012704,0.013914,0.018754,0.008469,0.004235,0.015124,0.003025,0.003630
